In [1]:
import pandas as pd
friends_df = pd.read_csv("headwinds_twitter_friends.csv")




In [3]:
friends_df.head()

name      username  \
0         Yves Peirsman  yvespeirsman   
1  Maria Capelle Frantz  mariacfrantz   
2   Guillermo Del Tired      mazarbor   
3               shannon    shannonkao   
4   Starsoft - PAX West     WestonPDX   

                                         description                  website  \
0            Natural Language Processing at @nlptown  https://t.co/oc8Q3pcMZb   
1  Cartoonist & illustrator ★ Creator of The Chan...  https://t.co/CPwwxklPnO   
2  canzine 2019 🌱 susie wilson (she/her) // carto...  https://t.co/lhXJ2TWFpf   
3  programmer by day, very tired by night & in be...  https://t.co/meY4VPLEtF   
4  Video game maker. Working on Savior, a smooth ...  https://t.co/eXeceoDYoS   

   followers  favourites  
0        818         556  
1       4267        6206  
2        191       16332  
3       4769        4809  
4       3969        3376

In [4]:
friends_df.describe()

followers     favourites
count  3.600000e+02     360.000000
mean   3.859998e+04   10072.719444
std    2.677872e+05   26485.266450
min    5.000000e+00       0.000000
25%    8.370000e+02     758.000000
50%    3.297000e+03    2825.500000
75%    1.103075e+04    9254.000000
max    4.119520e+06  381356.000000

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
import twitter
api = twitter.Api(consumer_key=os.environ['consumer_key'],
                  consumer_secret=os.environ['consumer_secret'],
                  access_token_key=os.environ['access_token_key'],
                  access_token_secret=os.environ['access_token_secret'])

In [6]:
result = api.GetFriends(user_id=None, screen_name=None, cursor=None, count=None, total_count=None, skip_status=False, include_user_entities=True)
# print(result)

# this could be useful to compare users to see who shares the same friends and discover who else I could befriend

In [8]:
len(result)

2034

In [10]:
paged = api.GetFriendsPaged(user_id=None, screen_name=None, cursor=-1, count=10, skip_status=False, include_user_entities=True)
print(paged)

(1645688340787488947, 0, [User(ID=114216916, ScreenName=shakasom), User(ID=744705937592594432, ScreenName=codeability), User(ID=101624176, ScreenName=steveschoger), User(ID=2353, ScreenName=davidcrow), User(ID=24338581, ScreenName=jamison_dance), User(ID=79005145, ScreenName=reimertz), User(ID=123037520, ScreenName=SiriAgrell), User(ID=239648605, ScreenName=PixelJunkNews), User(ID=18022416, ScreenName=sebmarkbage), User(ID=315636336, ScreenName=dribnet)])


In [13]:
from birdy.twitter import UserClient
client = UserClient(os.environ['consumer_key'],
                    os.environ['consumer_secret'],
                    os.environ['access_token_key'],
                    os.environ['access_token_secret'])


response = client.api['friends/list'].get()

TwitterRateLimitError: Too many requests (GET https://api.twitter.com/1.1/friends/list.json)

In [27]:
# print(response.data.get("users"))
# Unlike twitter, Birdy get the friends list 1 page at time but has deep user info while twitter is limited to user_id and screen_name
total_users = len(response.data.get("users"))
print("total users: ", total_users)

# print(response.data)
# 'next_cursor': 1645012985738854077, 'next_cursor_str': '1645012985738854077', 'previous_cursor': 0, 'previous_cursor_str'
# in order to get all the pages, I need to keep calling next page until next_cursor = 0
# due to rate limiting, I may not be all to get all the pages in one go.... 


total users:  20


In [75]:
import json

'''
next_cursor_page_19 = 1625030373660125000

response_page_19 = client.api['friends/list'].get(cursor=next_cursor_page_19)
json_data = response_page_19.data

with open('data/friends19.json', 'w') as outfile:
    json.dump(json_data, outfile)
'''    
    
def load_another_page(page_num):
    # need to get the next cursor from the previous page
    prev_page_num = page_num - 1
    with open('data/friends' + str(prev_page_num) + '.json') as json_file:
        loaded_data = json.load(json_file)
        next_cursor = loaded_data.get('next_cursor')
        print('building json page {}'.format(page_num))
        try:
            response = client.api['friends/list'].get(cursor=next_cursor)
            with open('data/friends' + str(page_num) + '.json', 'w') as outfile:
                json.dump(response.data, outfile)
            return True    
        except:
            print ("caught limit error on page {}".format(page_num))  
            return False

# load_another_page(20)

import time

# if hit a rate limit error - then need to wait another 15 before calling again... so let's say 20 minutes

def load_all_pages(start_page_num, max_page_num):
    start = start_page_num
    max = max_page_num
    no_error = True 
    while start < max and no_error:
        no_error = load_another_page(start)
        start = start + 1
        time.sleep(5)
    # I want to wait 20 minutes before attempting to call this again and pick up where I left off
    #while start < max and no_error is False:
    #    rate_limit_start = start - 1
    #    load_all_pages(rate_limit_start,max_page_num=102)  
    #    time.sleep(20 * 60)
          
load_all_pages(start_page_num=88,max_page_num=101)        
    

building json page 88
building json page 89
building json page 90
building json page 91
building json page 92
building json page 93
building json page 94
building json page 95
building json page 96
building json page 97
building json page 98
building json page 99
building json page 100
building json page 101


In [39]:
# I have 2000+ friends so how many pages do I need?


In [55]:
response = client.api['users/show'].get(screen_name="headwinds")
# statuses_count, retweet_count, friends_count, followers_count

# >>> format_dictionary = {'births': 'five', 'deaths': 'three'}
# >>> print "there are {births} births, and {deaths} deaths".format(**format_dictionary)

format_dictionary = {'followers_count': response.data.followers_count, 
                     'friends_count': response.data.friends_count, 
                     'statuses_count': response.data.statuses_count}

print ("headwinds has {friends_count} friends, {followers_count} followers, and {statuses_count} tweets".format(**format_dictionary))
totals_friends_pages = response.data.friends_count / 20;
print (totals_friends_pages)



headwinds has 2036 friends, 652 followers, and 3122 tweets
101.8
